# Optimizing SpMV

In this tutorial, we focus on sparse matrix-vector multiplication. A basic version of the code in Python/NumPy syntax is the following:

In [10]:
import dace
import numpy as np

from dace import dtypes


dtype = dace.float32
ntype = np.float32


M, N = (dace.symbol(s, dtype=dace.int32) for s in ('M', 'N'))
nnz = dace.symbol('nnz', dtype=dace.int64)


@dace.program(auto_optimize=True, device=dtypes.DeviceType.CPU)
def spmv(A_row: dace.uint32[M+1],
         A_col: dace.uint32[nnz],
         A_val: dtype[nnz],
         x: dtype[N],
         y: dtype[M]):

    for i in range(A_row.size - 1):
        cols = A_col[A_row[i]:A_row[i + 1]]
        vals = A_val[A_row[i]:A_row[i + 1]]
        y[i] = vals @ x[cols]

The above program can be executed in the following manner:

In [11]:
from numpy.random import default_rng
rng = default_rng(42)

M = 10000
N = 10000
nnz = (M * N) // 100
density = 0.01

x = rng.random((N,), dtype=ntype)
y = np.ndarray((M,), dtype=ntype)

from scipy.sparse import random
matrix = random(M,
                N,
                density=density,
                format='csr',
                dtype=ntype,
                random_state=rng)

rows = np.uint32(matrix.indptr).copy()
cols = np.uint32(matrix.indices).copy()
vals = matrix.data.copy()

assert(len(vals) == nnz)

with dace.config.set_temporary('profiling', value=True):
    spmv(rows, cols, vals, x, y)

ref = matrix @ x
assert(np.allclose(y, ref))


Promoted scalars {__tmp0, __tmp1, __tmp3, __tmp4} to symbols.
Applied 16 StateFusion, 1 RedundantSecondArray.
Applied 1 StateFusion.
Applied 1 LoopToMap.
Applied 5 StateFusion, 2 StateAssignElimination.
Automatically expanded library node "_MatMult_dot" with implementation "MKL".
Specializing the SDFG for symbols {'M': 10000, 'nnz': 1000000, 'N': 10000}
Statically allocating 1 transient arrays


/home/user/dace/dace/sdfg/nodes.py:637: UserWarning: loop_body maps to unused symbol(s): {'N', 'nnz', 'M'}
  warnings.warn(
/home/user/dace/dace/sdfg/sdfg.py:1786: UserWarning: SDFG "spmv" is already loaded by another object, recompiling under a different name.
  warnings.warn('SDFG "%s" is already loaded by another object, '
/home/user/dace/dace/codegen/targets/cpu.py:386: UserWarning: Variable-length array cols with size ((- __tmp3) + __tmp4) detected and was allocated on heap instead of StorageType.Register
  warnings.warn(
/home/user/dace/dace/codegen/targets/cpu.py:386: UserWarning: Variable-length array __tmp6 with size ((- __tmp3) + __tmp4) detected and was allocated on heap instead of StorageType.Register
  warnings.warn(


-- Configuring done
-- Generating done
-- Build files have been written to: /home/user/dace/tutorials/.dacecache/spmv/build

Scanning dependencies of target dacestub_spmv_2
[ 25%] Building CXX object CMakeFiles/dacestub_spmv_2.dir/tools/dacestub.cpp.o
[ 50%] Linking CXX shared library libdacestub_spmv_2.so
[ 50%] Built target dacestub_spmv_2
Scanning dependencies of target spmv_2
[ 75%] Building CXX object CMakeFiles/spmv_2.dir/home/user/dace/tutorials/.dacecache/spmv/src/cpu/spmv_2.cpp.o
/home/user/dace/tutorials/.dacecache/spmv/src/cpu/spmv_2.cpp: In function ‘void loop_body_0_0_0(spmv_2_t*, unsigned int*, unsigned int*, float*, float*, float*, long long int)’:
/home/user/dace/tutorials/.dacecache/spmv/src/cpu/spmv_2.cpp:41:38: warning: comparison of integer expressions of different signedness: ‘int’ and ‘unsigned int’ [-Wsign-compare]
   41 |             for (auto __i0 = 0; __i0 < ((- __tmp3) + __tmp4); __i0 += 1) {
      |                                 ~~~~~^~~~~~~~~~~~~~~~~~~~~~

To optimize SpMV, we rewrite the above basic code to simplify the transformations that will be applied:

In [12]:
@dace.program()
def spmv(A_row: dace.uint32[M+1],
         A_col: dace.uint32[nnz],
         A_val: dtype[nnz],
         x: dtype[N],
         y: dtype[M]):

    y[:] = 0.0
    for i in range(A_row.size - 1):
        start = A_row[i]
        finish = A_row[i+1]
        cols = A_col[start:finish]
        vals = A_val[start:finish]
        y[i] = dace.reduce(lambda x, y: x + y, vals * x[cols]) 

We optimize the above program for CPU as follows:

In [13]:
from dace.transformation.dataflow import (MapFusion,
                                          MapReduceFusion,
                                          TrivialTaskletElimination)
from dace.transformation.interstate import LoopToMap

def find_map_by_name(sdfg: dace.SDFG, name: str) -> dace.nodes.MapEntry:
    """ Finds the first map entry node by the given parameter name. """
    return next((n, s) for n, s in sdfg.all_nodes_recursive()
        if isinstance(n, dace.nodes.MapEntry) and n.label == name)

# Parse the program to an SDFG with simplification
cpu_sdfg = spmv.to_sdfg(simplify=True)
# Transform the loop over the i-variable to a Map
cpu_sdfg.apply_transformations(LoopToMap, {'itervar': 'i'})
cpu_sdfg.simplify()
# Fuse the multiplication Map (`vals * x[cols]`) with the reduction (`dace.reduce`)
cpu_sdfg.apply_transformations(MapReduceFusion)
# Fuse the above (multiplication) Map with the indirection Map (`x[cols]`)
fme, state = find_map_by_name(cpu_sdfg, 'indirect_slice')
fmx = state.exit_node(fme)
sme, state = find_map_by_name(cpu_sdfg, '_Mult__map')
MapFusion.apply_to(state.parent,
                   first_map_exit=fmx,
                   array=state.out_edges(fmx)[0].dst,
                   second_map_entry=sme)
# Remove trivial Tasklet
cpu_sdfg.apply_transformations(TrivialTaskletElimination)
cpu_sdfg.simplify()
# Since the outer Map iterates over the i-variable, all the WCRs on `y[i]` are
# executed by a single thread. Therefore, we do not need to use atomics.
for e, _ in cpu_sdfg.all_edges_recursive():
    if isinstance(e.data, dace.Memlet) and e.data.wcr:
        e.data.wcr_nonatomic = True
# Show SDFG
cpu_sdfg

Promoted scalars {finish, start} to symbols.
Applied 17 StateFusion, 2 SymbolAliasPromotion, 2 RedundantSecondArray, 2 StateAssignElimination.
Applied 1 LoopToMap.
Applied 3 StateFusion.
Applied 1 MapReduceFusion.
Applied 1 TrivialTaskletElimination.
Applied 1 RedundantArray.


We execute the optimized SDFG on CPU:

In [14]:
with dace.config.set_temporary('profiling', value=True):
    cpu_sdfg(A_row=rows, A_col=cols, A_val=vals, x=x, y=y)

assert(np.allclose(y, ref))

-- Configuring done
-- Generating done
-- Build files have been written to: /home/user/dace/tutorials/.dacecache/spmv/build

Scanning dependencies of target dacestub_spmv_3
[ 25%] Building CXX object CMakeFiles/dacestub_spmv_3.dir/tools/dacestub.cpp.o
[ 50%] Linking CXX shared library libdacestub_spmv_3.so
[ 50%] Built target dacestub_spmv_3
Scanning dependencies of target spmv_3
[ 75%] Building CXX object CMakeFiles/spmv_3.dir/home/user/dace/tutorials/.dacecache/spmv/src/cpu/spmv_3.cpp.o
/home/user/dace/tutorials/.dacecache/spmv/src/cpu/spmv_3.cpp: In function ‘void loop_body_0_1_0(spmv_3_t*, unsigned int*, unsigned int*, float*, float*, float*, long long int)’:
/home/user/dace/tutorials/.dacecache/spmv/src/cpu/spmv_3.cpp:18:38: warning: comparison of integer expressions of different signedness: ‘int’ and ‘unsigned int’ [-Wsign-compare]
   18 |             for (auto __i0 = 0; __i0 < (finish - start); __i0 += 1) {
      |                                 ~~~~~^~~~~~~~~~~~~~~~~~
[100%] L

We optimize the same program for GPU as follows:

In [15]:
# Copy CPU SDFG
import copy
gpu_sdfg = copy.deepcopy(cpu_sdfg)
# Move all input and output data directly to the GPU for benchmarking purposes
def copy_to_gpu(sdfg):
    for k, v in sdfg.arrays.items():
        if not v.transient and isinstance(v, dace.data.Array):
            v.storage = dace.dtypes.StorageType.GPU_Global
copy_to_gpu(gpu_sdfg)
# GPU Transform SDFG and auto-optimize
from dace.transformation.auto import auto_optimize
gpu_sdfg = auto_optimize.auto_optimize(gpu_sdfg, device=dtypes.DeviceType.GPU)
# Warp-tile the internal Map
from dace.transformation.dataflow import WarpTiling
me, state = find_map_by_name(gpu_sdfg, 'single_state_body_map')
me.schedule = dtypes.ScheduleType.GPU_Device
WarpTiling.apply_to(state.parent, mapentry=me)
# Show SDFG
gpu_sdfg

Applied 2 StateFusion.
Applied 1 GPUTransformSDFG.


We execute the optimized SDFG on GPU:

In [ ]:
import cupy
grows = cupy.asarray(rows)
gcols = cupy.asarray(cols)
gvals = cupy.asarray(vals)
gx = cupy.asarray(x)
gy = cupy.ndarray((M,), dtype=ntype)

with dace.config.set_temporary('profiling', value=True):
    gpu_sdfg(A_row=grows, A_col=gcols, A_val=gvals, x=gx, y=gy)

assert(np.allclose(gy, ref))

Below, we compare the performance of the above 3 programs against MKL (oneAPI 2021.3.0) and cuSPARSE (CUDA 11.4) on a machine with 2x Intel 6140 @ 2.30 GHz CPUs (total of 36 cores) and an Nvidia V100 (32GB PCIe) GPU.

Both DaCe CPU programs are executed with `OMP_NUM_THREADS=36`. The basic DaCe CPU program is calling MKL to compute the dot product, with `MKL_NUM_THREADS=1`. The MKL and cuSPARSE codes are given at the end of this tutorial. The GPU runtimes **do not** include copying data to/from the device.

The parameters are `M = N = 100,000` and variable `nnz` so that the density of the matrix is 10%, 1%, and 0.1%. The matrices are initialized with random data in a similar manner as above, so that the non-zero elements are uniformly distributed across the rows.

All the runtimes in the following table are in ms and correspond to the median out of 100 executions.


|Density|MKL|DaCe CPU Basic|DaCe CPU Optimized|cuSPARSE|DaCe GPU Optimized|
|------:|--:|-------------:|-----------------:|-------:|-----------------:|
|10%|85.258|84.638|87.628|12.174|11.152|
|1%|7.550|10.945|7.881|2.020|1.887|
|0.1%|0.673|1.196|0.791|0.247|0.261|


## MKL code

```CPP
extern "C" void spmv(int * __restrict__ a_row,
                     int * __restrict__ a_col,
                     float * __restrict__ a_val,
                     float *  __restrict__ x,
                     float *  __restrict__  b) {
                         
    sparse_matrix_t csrA;
    mkl_sparse_s_create_csr(&csrA, SPARSE_INDEX_BASE_ZERO, M, N,
                            a_row, a_row + 1, a_col, a_val);
    struct matrix_descr descrA_matrix;
    descrA_matrix.type = SPARSE_MATRIX_TYPE_GENERAL;
    descrA_matrix.mode = SPARSE_FILL_MODE_UPPER;
    descrA_matrix.diag = SPARSE_DIAG_NON_UNIT;

    mkl_sparse_s_mv(SPARSE_OPERATION_NON_TRANSPOSE,
                    1.0f, csrA, descrA_matrix, x, 0.0f, b);

}
```

## cuSPARSE code

```CPP
extern "C" void spmv(cusparseHandle_t h,
                     int M, int N, int nnz,
                     float * __restrict__ Aval,
                     int * __restrict__ Arow,
                     int * __restrict__ Acol,
                     float * __restrict__ x
                     float * __restrict__ b) {

    float alpha = 1.0f, beta = 0.0f;
    cusparseSpMatDescr_t matA;
    cusparseDnVecDescr_t vecX, vecY;
    void*                dBuffer    = NULL;
    size_t               bufferSize = 0;

    // Create sparse matrix A in CSR format
    cusparseCreateCsr(&matA, M, N, nnz,
                      Arow, Acol, Aval,
                      CUSPARSE_INDEX_32I, CUSPARSE_INDEX_32I,
                      CUSPARSE_INDEX_BASE_ZERO, CUDA_R_32F);

    // Create dense vector X
    cusparseCreateDnVec(&vecX, N, x, CUDA_R_32F);

    // Create dense vector y
    cusparseCreateDnVec(&vecY, M, b, CUDA_R_32F);

    // Allocate an external buffer if needed
    cusparseSpMV_bufferSize(h, CUSPARSE_OPERATION_NON_TRANSPOSE,
                            &alpha, matA, vecX, &beta, vecY, CUDA_R_32F,
                            CUSPARSE_SPMV_ALG_DEFAULT, &bufferSize);
    cudaMalloc(&dBuffer, bufferSize);

    // Execute SpMV
    cusparseSpMV(h, CUSPARSE_OPERATION_NON_TRANSPOSE,
                 &alpha, matA, vecX, &beta, vecY, CUDA_R_32F,
                 CUSPARSE_SPMV_ALG_DEFAULT, dBuffer);

    // Destroy matrix/vector descriptors
    cusparseDestroySpMat(matA);
    cusparseDestroyDnVec(vecX);
    cusparseDestroyDnVec(vecY);

    // Free extenral buffer
    cudaFree(dBuffer);

}
```